In [ ]:
import numpy as np
from skimage import io
from os.path import expanduser
from tqdm import tqdm
HOME = expanduser("~")
import os, sys
import cv2
import SimpleITK as sitk
from collections import OrderedDict
import subprocess
%load_ext autoreload
%autoreload 2

In [ ]:
PATH = '/home/eddyod/programming/pipeline_utility'
sys.path.append(PATH)
from utilities.sqlcontroller import SqlController
from utilities.alignment_utility import convert_resolution_string_to_um, SCALING_FACTOR

In [ ]:
animal = 'DK39'
DIR = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'
INPUT = os.path.join(DIR, 'CH1', 'thumbnail_cleaned')
OUTPUT = os.path.join(DIR, 'CH1', 'thumbnail_aligned')
ELASTIX = os.path.join(DIR, 'elastix')

In [ ]:
def register_with_masks(fixed, moving):
    #fixed = sitk.ReadImage('#######/imgRef.png', sitk.sitkFloat32);
    #moving = sitk.ReadImage('#######/imgRep.png', sitk.sitkFloat32)
    #maskFixed = sitk.ReadImage('#######/maskRef.png', sitk.sitkUInt8)
    #maskMoving= sitk.ReadImage('#######/maskRep.png', sitk.sitkUInt8)

    # Handle optimizer
    R = sitk.ImageRegistrationMethod()

    # Restrict the evaluation of the similarity metric thanks to masks
    #R.SetMetricFixedMask(maskFixed)
    #R.SetMetricMovingMask(maskMoving)

    # Set metric as mutual information using joint histogram
    R.SetMetricAsMattesMutualInformation(numberOfHistogramBins=255)

    # Gradient descent optimizer
    R.SetOptimizerAsRegularStepGradientDescent(learningRate=0.01, minStep=1e-5, 
                                               numberOfIterations=100, gradientMagnitudeTolerance=1e-8)

    #R.SetOptimizerScalesFromPhysicalShift()

    R.SetMetricSamplingStrategy(R.REGULAR) #R.RANDOM

    # Define the transformation (Rigid body here)

    transfo = sitk.CenteredTransformInitializer(fixed, moving, sitk.Euler2DTransform())

    R.SetInitialTransform(transfo)

    # Define interpolation method
    R.SetInterpolator(sitk.sitkLinear)

    # Add command to the registration process
    R.AddCommand(sitk.sitkIterationEvent, lambda: command_iteration(R))
    R.AddCommand(sitk.sitkStartEvent, lambda: start_plot())
    R.AddCommand(sitk.sitkEndEvent, lambda: end_plot())
    R.AddCommand(sitk.sitkIterationEvent, lambda: current_plot(R))
    # Perform registration
    final_transform = R.Execute(fixed, moving)
    return final_transform

    print(outTx)
    print("--------")
    print("Optimizer stop condition: {0}".format(R.GetOptimizerStopConditionDescription()))
    print("Number of iterations: {0}".format(R.GetOptimizerIteration()))
    print("--------")

    # Perform transformation and resample the moving image

    # Save transformation as tfm file
    sitk.WriteTransform(outTx, '/home/egs/f_nicolas/CODES_THESE/transfo_final.tfm')
    #sitk.Show(transfo.GetDisplacementField(),"Displacement field")

    # Resample moving image according to the last transformation
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed)
    resampler.SetInterpolator(sitk.sitkLinear)
    #resampler.SetDefaultPixelValue(100)
    resampler.SetTransform(outTx)
    out = resampler.Execute(moving)

In [ ]:
def register(fixed_image, moving_image, filter):
    """
    This code is very similar to the code from the ITK site. I added a variable
    for the filter but it should usually use the MOMENTS filter
    """
    
    initial_transform = sitk.CenteredTransformInitializer(
        fixed_image,
        moving_image,
        sitk.Euler2DTransform(),
        filter)

    REG = sitk.ImageRegistrationMethod()
    # Similarity metric settings.
    REG.SetMetricAsMattesMutualInformation(numberOfHistogramBins=150)
    REG.SetMetricSamplingStrategy(REG.REGULAR)
    REG.SetMetricSamplingPercentage(0.01)
    REG.SetInterpolator(sitk.sitkLinear)    
    # Optimizer settings.
    REG.SetOptimizerAsGradientDescent(learningRate=0.01, 
                                                      numberOfIterations=1000, 
                                                      convergenceMinimumValue=1e-6, 
                                                      convergenceWindowSize=10)
    REG.SetOptimizerScalesFromPhysicalShift()

    # Setup for the multi-resolution framework.            
    REG.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
    REG.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
    REG.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # Don't optimize in-place, we would possibly like to run this cell multiple times.
    REG.SetInitialTransform(initial_transform, inPlace=False)


    final_transform = REG.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                                                   sitk.Cast(moving_image, sitk.sitkFloat32))
    return final_transform

def create_matrix(final_transform):
    finalParameters = final_transform.GetParameters()
    fixedParameters = final_transform.GetFixedParameters()
    rot_rad, xshift, yshift = finalParameters
    center = np.array(fixedParameters)

    R = np.array([[np.cos(rot_rad), -np.sin(rot_rad)],
                  [np.sin(rot_rad), np.cos(rot_rad)]])
    shift = center + (xshift, yshift) - np.dot(R, center)
    T = np.vstack([np.column_stack([R, shift]), [0, 0, 1]])
    return T

def create_warp_transforms(animal, transforms, transforms_resol, resolution):
    #transforms_resol = op['resolution']
    transforms_scale_factor = convert_resolution_string_to_um(animal, resolution=transforms_resol) / convert_resolution_string_to_um(animal, resolution=resolution)
    tf_mat_mult_factor = np.array([[1, 1, transforms_scale_factor], [1, 1, transforms_scale_factor]])
    transforms_to_anchor = {
        img_name:
            convert_2d_transform_forms(np.reshape(tf, (3, 3))[:2] * tf_mat_mult_factor) for
        img_name, tf in transforms.items()}

    return transforms_to_anchor

def convert_2d_transform_forms(arr):
    """
    Just creates correct size matrix
    """
    return np.vstack([arr, [0,0,1]])



In [ ]:
moments = sitk.CenteredTransformInitializerFilter.MOMENTS
geometry = sitk.CenteredTransformInitializerFilter.GEOMETRY

In [ ]:
### loops through the image stack and performs the registration between each set of two consecutive files
### then saves the transformation in a file. It would probably be much better if we could just
### use the values from the final transform directly, or put in a dictionary instead of saving
### to file and then rereading them
files = sorted(os.listdir(INPUT))
transformation_to_previous_section = {}

for i in tqdm(range(1, len(files))):
    final_transform = None
    
    fixed_file = os.path.join(INPUT, files[i - 1])
    moving_file = os.path.join(INPUT, files[i])
    fixed_index = str(i-1).zfill(3)
    moving_index = str(i).zfill(3)
    moving_image = sitk.ReadImage(moving_file, sitk.sitkUInt16)
    fixed_image =  sitk.ReadImage(fixed_file, sitk.sitkUInt16)
    
    try:
        final_transform = register(fixed_image, moving_image, moments)
    except:
        print('Could not create moments transform for ', fixed_index,moving_index)
        try:
            final_transform = register(fixed_image, moving_image, geometry)
        except:
            print('Could not create geometry transform for ', fixed_index,moving_index)
                    
    if final_transform is not None:
        T = create_matrix(final_transform)
        transformation_to_previous_section[i] = T
    #sitk.WriteTransform(final_transform, outpath)

In [ ]:
anchor_index = len(files) // 2 # middle section of the brain
transformation_to_anchor_section = {}

for moving_index in range(len(files)):
    if moving_index == anchor_index:
        transformation_to_anchor_section[files[moving_index]] = np.eye(3)
    elif moving_index < anchor_index:
        T_composed = np.eye(3)
        for i in range(anchor_index, moving_index, -1):
            T_composed = np.dot(np.linalg.inv(transformation_to_previous_section[i]), T_composed)
        transformation_to_anchor_section[files[moving_index]] = T_composed
    else:
        T_composed = np.eye(3)
        for i in range(anchor_index + 1, moving_index + 1):
            T_composed = np.dot(transformation_to_previous_section[i], T_composed)
        transformation_to_anchor_section[files[moving_index]] = T_composed


#### Set the resolution and create a dictionary of transforms

In [ ]:
resolution = 'thumbnail'
warp_transforms = create_warp_transforms(animal, transformation_to_anchor_section, 'thumbnail', resolution)

In [ ]:
sqlController = SqlController(animal)
width = sqlController.scan_run.width
height = sqlController.scan_run.height
max_width = int(width * SCALING_FACTOR)
max_height = int(height * SCALING_FACTOR)
bgcolor = 'black' # this should be black, but white lets you see the rotation and shift

##### This gets the dictionary of transforms and passes those parameters to imagemagick's convert tool to rotate, shift and crop the image

In [ ]:
#OUTPUT = "setme to some place where you can write files"
ordered_transforms = OrderedDict(sorted(warp_transforms.items()))
for file, arr in tqdm(ordered_transforms.items()):
    T = np.linalg.inv(arr)
    sx = T[0, 0]
    sy = T[1, 1]
    rx = T[1, 0]
    ry = T[0, 1]
    tx = T[0, 2]
    ty = T[1, 2]
    
    op_str = f" +distort AffineProjection {sx},{rx},{ry},{sy},{tx},{ty}"
    op_str += f' -crop {max_width}x{max_height}+0.0+0.0!'
    input_fp = os.path.join(INPUT, file)
    output_fp = os.path.join(OUTPUT, file)
    if os.path.exists(output_fp):
        continue

    cmd = f"convert {input_fp} +repage -virtual-pixel background -background {bgcolor} {op_str} -flatten -compress lzw {output_fp}"
    subprocess.run(cmd, shell=True)
